In [2]:
import torch
import torchvision.models as models
from torchvision import transforms
from PIL import Image
import matplotlib.pyplot as plt
import numpy as np
import torch.nn.functional as F

In [3]:
def visualize_feature_maps(model, input_tensor, layer_name):
    """Visualizes the feature maps of a specified convolutional layer."""
    model.eval()
    feature_maps = None

    def hook_feature(module, input, output):
        nonlocal feature_maps
        feature_maps = output.detach().cpu()

    layer = model._modules.get(layer_name)
    if layer is None:
        print(f"Warning: Layer '{layer_name}' not found in the model.")
        return

    hook = layer.register_forward_hook(hook_feature)
    output = model(input_tensor)
    hook.remove()

    if feature_maps is not None:
        num_feature_maps = feature_maps.shape[1]
        rows = int(np.sqrt(num_feature_maps))
        cols = (num_feature_maps + rows - 1) // rows

        fig, axes = plt.subplots(rows, cols, figsize=(12, 12))
        for i in range(num_feature_maps):
            ax = axes.flatten()[i]
            feature_map = feature_maps[0, i, :, :].numpy()
            ax.imshow(feature_map, cmap='viridis')
            ax.axis('off')
        for j in range(num_feature_maps, rows * cols):
            axes.flatten()[j].axis('off')
        plt.suptitle(f'Feature Maps of {layer_name}', fontsize=16)
        plt.tight_layout(rect=[0, 0.03, 1, 0.95])
        plt.show()

In [4]:
import torch
from torchvision.models import resnet34

model = resnet34(pretrained=False)
try:
    model.load_state_dict(torch.load("plant_disease_model.pth", map_location=torch.device('cpu')))
    print("Model loaded successfully as resnet34!")
except RuntimeError as e:
    print(f"Error loading as resnet34: {e}")

from torchvision.models import resnet50

model = resnet50(pretrained=False)
try:
    model.load_state_dict(torch.load("plant_disease_model.pth", map_location=torch.device('cpu')))
    print("Model loaded successfully as resnet50!")
except RuntimeError as e:
    print(f"Error loading as resnet50: {e}")

C:\Users\rishi\AppData\Local\Programs\Python\Python312\Lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
C:\Users\rishi\AppData\Local\Programs\Python\Python312\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)


Error loading as resnet34: Error(s) in loading state_dict for ResNet:
	Missing key(s) in state_dict: "conv1.weight", "bn1.weight", "bn1.bias", "bn1.running_mean", "bn1.running_var", "layer1.0.conv1.weight", "layer1.0.bn1.weight", "layer1.0.bn1.bias", "layer1.0.bn1.running_mean", "layer1.0.bn1.running_var", "layer1.0.conv2.weight", "layer1.0.bn2.weight", "layer1.0.bn2.bias", "layer1.0.bn2.running_mean", "layer1.0.bn2.running_var", "layer1.1.conv1.weight", "layer1.1.bn1.weight", "layer1.1.bn1.bias", "layer1.1.bn1.running_mean", "layer1.1.bn1.running_var", "layer1.1.conv2.weight", "layer1.1.bn2.weight", "layer1.1.bn2.bias", "layer1.1.bn2.running_mean", "layer1.1.bn2.running_var", "layer1.2.conv1.weight", "layer1.2.bn1.weight", "layer1.2.bn1.bias", "layer1.2.bn1.running_mean", "layer1.2.bn1.running_var", "layer1.2.conv2.weight", "layer1.2.bn2.weight", "layer1.2.bn2.bias", "layer1.2.bn2.running_mean", "layer1.2.bn2.running_var", "layer2.0.conv1.weight", "layer2.0.bn1.weight", "layer2.0.bn1.

In [5]:
# Define image transformations (must match your training)
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]) # Use your training normalization
    ])

In [6]:
# Load a sample image
image_path = 'Screenshot 2025-04-10 132915.png'
try:
    img = Image.open(image_path).convert('RGB')
    input_tensor = transform(img).unsqueeze(0)
except FileNotFoundError:
    print(f"Error: Image not found at {image_path}")
    exit()

In [ ]:
# Choose a convolutional layer to visualize
layer_to_visualize = 'layer2'  # For standard ResNet architectures

# Visualize the feature maps of your trained model
visualize_feature_maps(model, input_tensor, layer_to_visualize)